In [1]:
from shapely.geometry import Point
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import xlrd
import time
from IPython.display import Audio

In [2]:
#full runtime indicator start
full_time=time.time()

In [3]:
#finish indicator
wow = '../../Data/Wow.mp3'
finished=Audio(url = wow, autoplay = True) 

In [4]:
#notebook complete indicator
notebook = '../../Data/Hugo Good Job.mp3'
complete=Audio(url = notebook, autoplay = True)

In [5]:
map = gpd.read_file('../../Data/Taxi Zones Shape/taxi_zones.shp').to_crs('epsg:4326')
map = map.drop(['Shape_Area', 'Shape_Leng', 'OBJECTID'], axis=1)


complete

In [6]:
df=pd.read_csv('../../Data/Taxi Data/yellow_14/yellow_tripdata_2014-09.csv')

C:\Users\Bro Bro\.conda\envs\geospatial\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


df.info()

df2=df.head(100)

df2.columns

In [7]:
start_time = time.time()

df['dropoff_geometry'] = df.apply(lambda x: Point((float(x[' dropoff_longitude']), 
                                             float(x[' dropoff_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 867.3260481357574 seconds ---


In [8]:
start_time = time.time()

df['pickup_geometry'] = df.apply(lambda x: Point((float(x[' pickup_longitude']), 
                                             float(x[' pickup_latitude']))), 
                                             axis=1)

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 833.8491561412811 seconds ---


In [9]:
df.head(3)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,surcharge,mta_tax,tip_amount,tolls_amount,total_amount,dropoff_geometry,pickup_geometry
0,VTS,2014-09-10 12:19:00,2014-09-10 12:39:00,2,1.53,-73.96227,40.779057,1,NaN,-73.976247,40.759452,CSH,13.5,0.0,0.5,0.0,0.0,14.0,POINT (-73.976247 40.759452),POINT (-73.96227 40.779057)
1,VTS,2014-09-10 12:33:00,2014-09-10 12:41:00,1,0.50,-73.97403,40.762792,1,NaN,-73.978342,40.756787,UNK,6.0,0.0,0.5,1.2,0.0,7.7,POINT (-73.978342 40.756787),POINT (-73.97403 40.762792)
2,VTS,2014-09-10 12:31:00,2014-09-10 12:39:00,5,1.25,-73.98052,40.738287,1,NaN,-73.994565,40.729317,CSH,7.0,0.0,0.5,0.0,0.0,7.5,POINT (-73.99456499999998 40.729317),POINT (-73.98052 40.738287)


In [10]:
df2_start=df[[' passenger_count',' trip_distance',' fare_amount',' mta_tax',' tip_amount',' tolls_amount','pickup_geometry']]

In [11]:
df2_end=df[['dropoff_geometry']]

In [12]:
start_time = time.time()
df2_start=gpd.GeoDataFrame(df2_start,
                     crs=map.crs,
                     geometry=df2_start['pickup_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 112.3300244808197 seconds ---


In [13]:
start_time = time.time()
df2_end=gpd.GeoDataFrame(df2_end,
                     crs=map.crs,
                     geometry=df2_end['dropoff_geometry'])
print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 112.64537596702576 seconds ---


In [14]:
start_time = time.time()

geo_processing_start = gpd.sjoin(df2_start, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1127.4421677589417 seconds ---


In [15]:
start_time = time.time()

geo_processing_end = gpd.sjoin(df2_end, map, op = 'within')

print("--- %s seconds ---" % (time.time() - start_time))
finished

--- 1156.2665765285492 seconds ---


In [16]:
geo_processing_start

,passenger_count,trip_distance,fare_amount,mta_tax,tip_amount,tolls_amount,pickup_geometry,geometry,index_right,zone,LocationID,borough
0,2,1.53,13.5,0.5,0.00,0.0,POINT (-73.96227 40.779057),POINT (-73.96227 40.77906),42,Central Park,43,Manhattan
48,1,3.36,13.5,0.5,2.80,0.0,POINT (-73.972555 40.765042),POINT (-73.97255 40.76504),42,Central Park,43,Manhattan
53,2,1.34,6.0,0.5,1.30,0.0,POINT (-73.965772 40.778142),POINT (-73.96577 40.77814),42,Central Park,43,Manhattan
101,1,3.99,23.0,0.5,0.00,0.0,POINT (-73.97298000000001 40.76512),POINT (-73.97298 40.76512),42,Central Park,43,Manhattan
468,1,3.13,16.5,0.5,0.00,0.0,POINT (-73.963982 40.776735),POINT (-73.96398 40.77674),42,Central Park,43,Manhattan
...,...,...,...,...,...,...,...,...,...,...,...,...
6834046,1,1.70,11.0,0.5,2.40,0.0,POINT (-74.20487799999998 40.555167),POINT (-74.20488 40.55517),203,Rossville/Woodrow,204,Staten Island
8994444,1,0.00,12.0,0.5,2.40,0.0,POINT (-74.21629 40.54548),POINT (-74.21629 40.54548),203,Rossville/Woodrow,204,Staten Island
9916148,1,0.03,130.0,0.0,26.00,0.0,POINT (-74.20335 40.54639),POINT (-74.20335 40.54639),203,Rossville/Woodrow,204,Staten Island
9493684,3,1.46,6.5,0.5,1.40,0.0,POINT (-74.129625 40.637797),POINT (-74.12963 40.63780),186,Port Richmond,187,Staten Island


In [17]:
group_start = geo_processing_start.groupby(['LocationID']).describe()

In [18]:
group_start.columns = ['_'.join(col) for col in group_start.columns.values]

group_start.to_csv('../../Data/yellow_tripdata_2014-2-start-processed.csv')

In [19]:
group_end = geo_processing_end.value_counts(['LocationID'])

In [20]:
group_end = pd.DataFrame(group_end,columns=['dropoff_count'])

group_end.to_csv('../../Data/yellow_tripdata_2014-2-end-processed.csv')

In [21]:
merge=group_start.merge(group_end, how='left', on='LocationID')

In [22]:
#REMEMBER TO RENAME CSV BEFORE START
merge.to_csv('../../Data/yellow_tripdata_2014-09-processed.csv')
complete

In [23]:
print("--- %s seconds ---" % (time.time() - full_time))

--- 4263.77214384079 seconds ---


In [24]:
merge

,passenger_count_count,passenger_count_mean,passenger_count_std,passenger_count_min,passenger_count_25%,passenger_count_50%,passenger_count_75%,passenger_count_max,trip_distance_count,trip_distance_mean,...,tolls_amount_max,index_right_count,index_right_mean,index_right_std,index_right_min,index_right_25%,index_right_50%,index_right_75%,index_right_max,dropoff_count
LocationID,,,,,,,,,,,,,,,,,,,,,
1,807.0,1.410161,0.882162,0.0,1.0,1.0,2.00,7.0,807.0,1.986989,...,20.00,807.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20721.0
2,6.0,1.000000,0.000000,1.0,1.0,1.0,1.00,1.0,6.0,6.865000,...,0.00,6.0,1.0,0.0,1.0,1.0,1.0,1.0,1.0,10.0
3,28.0,1.464286,1.261455,1.0,1.0,1.0,1.00,5.0,28.0,4.208571,...,5.33,28.0,2.0,0.0,2.0,2.0,2.0,2.0,2.0,636.0
4,46108.0,1.672595,1.323189,0.0,1.0,1.0,2.00,9.0,46108.0,2.814047,...,19.50,46108.0,3.0,0.0,3.0,3.0,3.0,3.0,3.0,65702.0
5,4.0,1.250000,0.500000,1.0,1.0,1.0,1.25,2.0,4.0,21.900000,...,0.00,4.0,4.0,0.0,4.0,4.0,4.0,4.0,4.0,67.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
259,57.0,2.017544,1.586677,1.0,1.0,1.0,2.00,6.0,57.0,3.850877,...,5.33,57.0,258.0,0.0,258.0,258.0,258.0,258.0,258.0,1481.0
260,8368.0,1.722873,1.431197,0.0,1.0,1.0,2.00,6.0,8368.0,3.439411,...,18.33,8368.0,259.0,0.0,259.0,259.0,259.0,259.0,259.0,17215.0
261,61985.0,1.714076,1.335387,0.0,1.0,1.0,2.00,6.0,61985.0,4.118050,...,19.55,61985.0,260.0,0.0,260.0,260.0,260.0,260.0,260.0,67071.0


In [25]:
group_start.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 259 entries, 1 to 263
Data columns (total 56 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0    passenger_count_count  259 non-null    float64
 1    passenger_count_mean   259 non-null    float64
 2    passenger_count_std    256 non-null    float64
 3    passenger_count_min    259 non-null    float64
 4    passenger_count_25%    259 non-null    float64
 5    passenger_count_50%    259 non-null    float64
 6    passenger_count_75%    259 non-null    float64
 7    passenger_count_max    259 non-null    float64
 8    trip_distance_count    259 non-null    float64
 9    trip_distance_mean     259 non-null    float64
 10   trip_distance_std      256 non-null    float64
 11   trip_distance_min      259 non-null    float64
 12   trip_distance_25%      259 non-null    float64
 13   trip_distance_50%      259 non-null    float64
 14   trip_distance_75%      259 non-null    fl